In [36]:
import numpy as np
from sklearn.decomposition import PCA, FastICA
from sklearn.metrics import mean_squared_error

def generate_signals(t, noise_level):
    n1 = np.random.normal(0, 1, len(t))
    n2 = np.random.normal(0, 1, len(t))
    n3 = np.random.normal(0, 1, len(t))
    f1 = np.sin(t) + 0.001 * n1 * noise_level
    f2 = 2 * (t - np.floor(t)) + 0.002 * n2 * noise_level
    f3 = 0.01 * t + 0.001 * n3 * noise_level
    return np.vstack((f1, f2, f3))

# Function to measure recovery accuracy
def correlation(true, estimated):
    return [pearsonr(true[i], estimated[i])[0] for i in range(3)]

for level in np.linspace(0.5, 5, 10):
    t = np.linspace(0, 100, 1000)
    A = np.random.rand(3, 3)
    f = generate_signals(t, level)
    x = A @ f
    x = x.T

    # PCA und ICA
    pca = PCA(n_components=3)
    ica = FastICA(n_components=3)

    # Fit and transform
    pca_transformed = pca.fit_transform(x)
    ica_transformed = ica.fit_transform(x)

    # PCA Reconstruction
    pca_reconstructed = pca.inverse_transform(pca_transformed).T

    # ICA Reconstruction
    ica_reconstructed = ica.inverse_transform(ica_transformed).T

    # MSE-Berechnung für jedes Signal
    mse_pca = [mean_squared_error(f[i], pca_reconstructed[i]) for i in range(3)]
    mse_ica = [mean_squared_error(f[i], ica_reconstructed[i]) for i in range(3)]

    print("Noise level ", level)
    print("PCA MSE: ", mse_pca)
    print("ICA MSE: ", mse_ica)


Noise level  0.5
PCA MSE:  [1.670364569124933, 0.5730870809298607, 0.3521204546453268]
ICA MSE:  [1.6703645691249331, 0.5730870809298608, 0.3521204546453271]
Noise level  1.0
PCA MSE:  [0.5401766061270433, 0.05381002246750201, 0.07211557641965624]
ICA MSE:  [0.5401766061270434, 0.053810022467501856, 0.07211557641965617]
Noise level  1.5
PCA MSE:  [1.1039620654329, 0.23767288009452045, 0.8445817559636152]
ICA MSE:  [1.1039620654329019, 0.23767288009452056, 0.8445817559636153]
Noise level  2.0
PCA MSE:  [1.2700631353408178, 0.7870095510853677, 0.706284429634446]
ICA MSE:  [1.2700631353408185, 0.7870095510853677, 0.7062844296344454]
Noise level  2.5
PCA MSE:  [0.7505922294440047, 0.10139395771018476, 0.36134804501309586]
ICA MSE:  [0.7505922294440048, 0.10139395771018442, 0.36134804501309564]
Noise level  3.0
PCA MSE:  [2.0904699238699633, 0.28283101767503055, 0.6273261104112599]
ICA MSE:  [2.0904699238699633, 0.2828310176750305, 0.6273261104112596]
Noise level  3.5
PCA MSE:  [0.395830076

/Users/robinthielking/miniconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/Users/robinthielking/miniconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/Users/robinthielking/miniconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/Users/robinthielking/miniconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/Users/robinthielking/miniconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/User

https://stackoverflow.com/questions/36566844/pca-projection-and-reconstruction-in-scikit-learn